# Entrega 2 de Robotica Computacional

## Calculando a 'Constante de distancia'
A constante de distancia é o que eu estou chamando a constante que dividida pelo tamanho do objeto em pixels, vezes seu diametro real vai resultar na distancia da camera ao objeto.

In [15]:
diametro_real = 2.8
distancia_real = 65.5
diametro_pixels = 30

diametro_pixels*(distancia_real/diametro_real)

701.7857142857143

## Importando as bibliotecas necessarias

In [16]:
import cv2
import numpy as np
from __future__ import print_function

### Criando a 'mira'
A mira é a regiao da qual a camera vai tirar as cores que voce quer procurar, o proximo bloco de codigo torna a mira funcional.

In [17]:
# Ponto esquerdo superior da 'mira'
ponto1 = (300,200)
# Ponto inferior direito da 'mira'
ponto2 = (320,220)

# Valores maximos dos canais hsv
high = np.array([300,300,300])
# Valores minimos dos canais hsv
low = np.array([0,0,0])

# Contador de 'ciclos'
time = 0

# Fix é um valor adicionado ao maximo e subtraido do minimo na mira, para ter certeza de que todo o objeto é capturado no
# mask
fix = 20

### Finalmente printando a distancia
O proximo bloco de codigo torna possivel printar a distancia de um circulo de 2.8cm de (eu acho, nao testei) qualquer cor da camera.

In [19]:
# Função que ajusta a cor sendo filtrada no mask para as cores dentro da mira
def reset():
    for idx in range(0,3):
        low[idx] += (hsv[ponto1[1]:ponto2[1],ponto1[0]:ponto2[0],idx].min() - low[idx]) - fix
        high[idx] += (hsv[ponto1[1]:ponto2[1],ponto1[0]:ponto2[0],idx].max() - high[idx]) + fix

# Esse for é necessario para a minha camera nao dar pau
for x in [1,0]:
    webcam = cv2.VideoCapture(x)

# While pra fazer a captura de imagem funcionar
while True:
    time += 1
    
    val, image = webcam.read()
    
    # Mudando os canais da imagem para hsv
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Chamando a função reset()
    reset()
    
    # Usando mask para filtrar a imagem hsv usando os valores tirados da mira
    mask = cv2.inRange(hsv, low, high)
    
    # Desenhando os contornos no mask
    contornos, arvore = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    
    if len(contornos) >= 1:
        # Definindo as variaveis maior_contorno_area e maior_contorno para nao dar 'variavel nao definida'
        maior_contorno_area = cv2.contourArea(contornos[0])
        maior_contorno = contornos[0]

        # Achando o maior contorno na imagem
        for cnt in contornos:
            if (cv2.contourArea(cnt) > maior_contorno_area):
                maior_contorno = cnt
                maior_contorno_area = cv2.contourArea(cnt)

        denominador = (maior_contorno[:,0,0].max()-maior_contorno[:,0,0].min())
                
        if denominador > 0:
            # Printar a distancia do objeto à tela
            print((701.7857142857143/denominador)*2.8,end='\r')

        # Desenhar o maior contorno na tela
        cv2.drawContours(image, maior_contorno, -1, [255, 0, 255], 5)
    
    # Desenhar a 'mira' na tela
    cv2.rectangle(image,ponto1,ponto2,(0,255,0),3)
    
    #Mostrar imagem e mask
    cv2.imshow('Image',image)
    
    ##### AJUDA, O MASK NAO TA SENDO MOSTRADO DIREITO #####
    cv2.imshow('Mask',mask)

    # Fechar janelas se q for precionado, nao essa linha mas isso sai do while e sair do while fecha as janela
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release() # fecha a webcam
cv2.destroyAllWindows()